In [11]:
import processors
print(processors.__version__)

3.0.3


In [13]:
%reload_ext autoreload
%autoreload 2

In [5]:
from processors import *
from processors.visualization import JupyterVisualizer as viz

API = ProcessorsAPI(port=8881, keep_alive=True)

INFO - Using path given via $PROCESSORS_SERVER
INFO - Connection with server established!
INFO - Server version meets recommendations (v3.0.2)


In [14]:
import os

# abstracts_dir = "/Users/gus/repos/ohsu-hackathon/OHSU_NLP_Hackathon/sentences_and_abstracts/CIViC_abstracts"
abstracts_dir = "/Users/manningh/PycharmProjects/OHSU_NLP_Hackathon/sentences_and_abstracts/CIViC_abstracts"
serialized_documents_dir = "/Users/manningh/PycharmProjects/odin-drug-outcomes/sensitivity-and-adverse-outcomes/serialized-abstracts"

def annotate_documents(collection_dir):
    for f in os.listdir(collection_dir):
        file_path = os.path.join(collection_dir, f)
        text = open(file_path).read()
        doc = API.bionlp.annotate(text) 
        # use filename as the doc's id
        doc.id = os.path.splitext(f)[0]
        yield doc

In [15]:
#docs = annotate_documents(abstracts_dir)

In [16]:
def serialize_documents(collection):
    for doc in collection:
        #doc.id = os.path.splitext(doc.id)[0]
        with open("{}.json".format(doc.id), "w") as out_file:
            out_file.write(doc.to_JSON())

In [17]:
#serialize_documents(docs)

In [18]:
import json

def deserialize_doc_collection(serialized_docs_dir):
    for json_file in os.listdir(serialized_docs_dir):
        fpath = os.path.join(serialized_docs_dir, json_file)
        with open(fpath) as infile:
            text = infile.read()
        doc_dict = json.loads(text)
        doc = Document.load_from_JSON(doc_dict)
        # FIXED, but not released (ADD TEST)
        # py-processors v3.0.3 wasn't retrieving the doc id
        doc.id = os.path.splitext(json_file)[0]
        yield doc

In [19]:
abstracts = list(deserialize_doc_collection(serialized_documents_dir))

In [20]:
rules = open("rules.yml", "r")

def extract_from_collection(collection):
    for doc in collection:
        mentions = API.odin.extract_from_document(doc, rules)
        for m in sorted(mentions, key=lambda x: (x.document.id, x.sentence, x.start), reverse=False):
            yield m

In [21]:
def display_mentions(mentions):
    for m in mentions:
        print(m.document.id)
        viz.display_mention(m)

In [22]:
mentions = list(extract_from_collection(abstracts))

In [ ]:
#API.stop_server()

In [15]:
from utils import *

drugs_file = "sensitivity-and-adverse-outcomes/drugs_one_per_line.txt"

drug_ngram_counts = ngrams_from_file(file_path=drugs_file, nlp_pipeline=API, n=3)

In [17]:
drug_ngram_counts.most_common()

[('ib$', 0.01766996106618748),
 ('nib', 0.011979634621144056),
 ('tin', 0.011829889188379754),
 ('ini', 0.010182689427972447),
 ('ne$', 0.004941599281221923),
 ('in$', 0.004492362982929021),
 ('ine', 0.0037436358191075174),
 ('^AZ', 0.0035938903863432167),
 ('mab', 0.0035938903863432167),
 ('ab$', 0.0035938903863432167),
 ('ati', 0.0035938903863432167),
 ('uma', 0.0032943995208146153),
 ('01$', 0.002994908655286014),
 ('AZD', 0.0028451632225217132),
 ('sta', 0.002545672356993112),
 ('iti', 0.002395926924228811),
 ('ide', 0.0022461814914645105),
 ('cin', 0.0020964360587002098),
 ('met', 0.0020964360587002098),
 ('eti', 0.0020964360587002098),
 ('^($', 0.0020964360587002098),
 ('ert', 0.001946690625935909),
 ('de$', 0.001946690625935909),
 ('oli', 0.001946690625935909),
 ('nti', 0.001946690625935909),
 ('ino', 0.001946690625935909),
 ('^)$', 0.001946690625935909),
 ('ani', 0.001946690625935909),
 ('rti', 0.001946690625935909),
 ('ost', 0.0017969451931716084),
 ('tib', 0.00179694519317160